In [1]:
import pandas as pd

# Visualization librairies
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mtick
from matplotlib.lines import Line2D

#Calculating libraries
from scipy.stats import bootstrap
import numpy as np
from scipy import stats
import scipy as sp
import matplotlib.dates as md

#statistical librairies
from os import stat

# Others
from functools import partial
from ast import literal_eval

# Import libraries
import requests
from bs4 import BeautifulSoup

In [2]:
character=pd.read_table('./data/character.metadata.tsv',header=None)
movies=pd.read_table('./data/movie.metadata.tsv',header=None)
names=pd.read_csv('./data/name.clusters.txt',sep="\t",header=None)
summaries=pd.read_csv('./data/plot_summaries.txt',sep="\t",header=None)
tvtropes=pd.read_csv('./data/tvtropes.clusters.txt',sep="\t",header=None)

In [3]:
character=character.rename(columns={0:'wikipedia_movie_id',1:'Freebase_movie_ID',3:'character_name',4:'Actor_DOB',5:'Actor_gender',6:'actor_height',7:'Actor_etnicity',8:'Actor_name',9:'Actor_age_at_movie_release',10:'Freebase_character_map'})
movies.rename(columns={0:'wikipedia_movie_id',1:'Freebase_movie_ID',2:'Movie_name',3:'Movie_release',4:'Box_office_revenue',5:'Movie_runtime',6:'Movie_language',7:'Movie_country',8:'Movie_genre'},inplace=True)
names.rename(columns={0:'Character_names',1:'Freebase_ID'},inplace=True)
tvtropes.rename(columns={0:'Character_type',1:'Freebase_ID'},inplace=True)
summaries.rename(columns={0:'wikipedia_movie_id',1:'Plot_summaries'},inplace=True)

In [4]:
tvtropes["dict_ID"]=[literal_eval(i) for i in tvtropes.Freebase_ID]
tvtropes=pd.concat([tvtropes.drop(['dict_ID'], axis=1), tvtropes
                    ['dict_ID'].apply(pd.Series)], axis=1)

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from nltk.tokenize import word_tokenize
import re

stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def stem_string(string):
    words = word_tokenize(string)
    lemmatized_words = [stemmer.stem(word) for word in words]
    return lemmatized_words

stem_string('biographical biography')

def clean_string(string):

    clean_s = string.lower()
    clean_s = clean_s.replace('/', ' ')

    clean_s = re.sub(r'-(?!.*(fi|white))', ' ', clean_s)
    # remove everything except alphabets 
    clean_s = re.sub("[^a-zA-Z-]"," ",clean_s) 
    # remove whitespaces 
    clean_s = ' '.join(clean_s.split()) 
    return clean_s

def clean(var):

    out = ''

    if type(var) == list:
        out = var.copy()
        for i,s in enumerate(var):
            out[i] = clean_string(s)
    else :
        if type(var) == dict:
            out = var.copy()
            for key, value in var.items():
                out[clean_string(key)] = clean(value)
        else:
            out = clean_string(var)
    return out

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nicte\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
import json

from collections import defaultdict

movies['Movie_release'] = pd.to_datetime(movies['Movie_release'], errors = 'coerce')
sorted_movies = movies.sort_values("Movie_release", ascending=True).dropna(subset=['Movie_release'])

genres_per_year = sorted_movies[['Movie_release', 'Movie_name', 'Movie_genre']]
genres_per_year['Movie_genre'] = genres_per_year.Movie_genre.apply(json.loads).apply(lambda x: list(x.values()))

genres_per_year.head(20)

C:\Users\nicte\AppData\Local\Temp/ipykernel_1372/616442467.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_per_year['Movie_genre'] = genres_per_year.Movie_genre.apply(json.loads).apply(lambda x: list(x.values()))


,Movie_release,Movie_name,Movie_genre
57963,1888-01-01,Roundhay Garden Scene,"[Short Film, Silent film, Indie, Black-and-white]"
38764,1889-01-01,"Leisurely Pedestrians, Open Topped Buses and Hansom Cabs with Trotting Horses","[Short Film, Silent film, Indie, Black-and-white]"
60249,1890-01-01,London's Trafalgar Square,"[Short Film, Silent film]"
53907,1890-01-01,"Monkeyshines, No. 1",[Silent film]
7486,1891-01-01,Men Boxing,"[Short Film, Silent film]"
25984,1891-01-01,Dickson Greeting,"[Short Film, Indie]"
17849,1891-01-01,Newark Athlete,[Short Film]
50565,1892-10-28,Pauvre Pierrot,"[Short Film, Silent film, Indie, Animation]"
14231,1892-10-28,Un bon bock,"[Short Film, Silent film, Indie, Animation]"
39307,1892-10-28,Le Clown et ses chiens,"[Short Film, Silent film, Indie, Animation]"


In [18]:


movies['Movie_release'] = pd.to_datetime(movies['Movie_release'], errors = 'coerce')
sorted_movies = movies.sort_values("Movie_release", ascending=True).dropna(subset=['Movie_release'])

genres_per_year = sorted_movies[['Movie_release', 'Movie_name', 'Movie_genre']]
genres_per_year['Movie_genre'] = genres_per_year.Movie_genre.apply(json.loads).apply(lambda x: clean(list(x.values())))

occur = genres_per_year.copy()

occur.head(20)

C:\Users\nicte\AppData\Local\Temp/ipykernel_1372/896153644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_per_year['Movie_genre'] = genres_per_year.Movie_genre.apply(json.loads).apply(lambda x: clean(list(x.values())))


,Movie_release,Movie_name,Movie_genre
57963,1888-01-01,Roundhay Garden Scene,"[short film, silent film, indie, black-and-white]"
38764,1889-01-01,"Leisurely Pedestrians, Open Topped Buses and Hansom Cabs with Trotting Horses","[short film, silent film, indie, black-and-white]"
60249,1890-01-01,London's Trafalgar Square,"[short film, silent film]"
53907,1890-01-01,"Monkeyshines, No. 1",[silent film]
7486,1891-01-01,Men Boxing,"[short film, silent film]"
25984,1891-01-01,Dickson Greeting,"[short film, indie]"
17849,1891-01-01,Newark Athlete,[short film]
50565,1892-10-28,Pauvre Pierrot,"[short film, silent film, indie, animation]"
14231,1892-10-28,Un bon bock,"[short film, silent film, indie, animation]"
39307,1892-10-28,Le Clown et ses chiens,"[short film, silent film, indie, animation]"


In [19]:
URL = 'https://www.studiobinder.com/blog/movie-genres-list/'

# Make the request
r = requests.get(URL) # /ip: Returns the requester's IP Address.

print('Response status code: {0}\n'.format(r.status_code))
print('Response headers: {0}\n'.format(r.headers))

Response status code: 200

Response headers: {'Date': 'Sat, 12 Nov 2022 20:53:52 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Last-Modified': 'Tue, 01 Nov 2022 11:34:24 GMT', 'Server': 'Studiobinder Blog', 'X-FastCGI-Cache': 'HIT', 'Strict-Transport-Security': 'max-age=63072000; ', 'Content-Encoding': 'gzip'}



In [20]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.h5.string

publications_wrappers = soup.find_all('div', {"data-css": ["tve-u-16d41491117", "tve-u-16d41ed6dc1", "tve-u-17317362207"]})

print('Total number of items: {0}'.format(len(publications_wrappers)))

genres = dict([])

for p in publications_wrappers:    
    sub_g = p.find_all('h5')
    major_genres = p.find_all('h3')

    for g in major_genres :

        m_genre = clean(g.string.rsplit(' ', 1)[0])
        genres[m_genre] = []
        genres[m_genre].append(clean(m_genre))

    for s in sub_g :
        genres[m_genre].append(clean(s.string))

genres.popitem()

print(genres)

Total number of items: 14
{'action': ['action', 'heroic bloodshed', 'military action', 'espionage', 'wuxia action', 'disaster', 'adventure', 'superhero'], 'animation': ['animation', 'traditional', 'stop motion', 'claymation', 'cutout', 'computer generated imagery', 'puppetry', 'live action'], 'comedy': ['comedy', 'action comedy', 'dark comedy black comedy', 'romantic comedy', 'buddy comedy', 'road comedy', 'slapstick comedy', 'parody', 'spoof', 'satire', 'sitcom', 'sketch comedy', 'mockumentary', 'prank'], 'crime': ['crime', 'caper', 'heist', 'gangster', 'cop police', 'detective', 'courtroom', 'procedural'], 'drama': ['drama', 'melodrama', 'teen drama', 'philosophical drama', 'medical drama', 'legal drama', 'political drama', 'anthropological drama', 'religious drama', 'docudrama'], 'experimental': ['experimental', 'surrealist', 'absurdist'], 'fantasy': ['fantasy', 'contemporary fantasy', 'urban fantasy', 'dark fantasy', 'high fantasy', 'myth'], 'historical': ['historical', 'historical

In [21]:
from utils.genres import additional_wordsets
additional_wordsets = additional_wordsets
def mergeDictionary(dict_1, dict_2):
   dict_3 = {**dict_1, **dict_2}
   for key, value in dict_3.items():
       if key in dict_1 and key in dict_2:
               dict_3[key] = value + dict_1[key]
   return dict_3

In [22]:
genre_map = mergeDictionary(genres, clean(additional_wordsets))
print(genre_map)

{'action': ['swashbuckle', 'combat', 'martial', 'boxing', 'sword', 'action', 'heroic bloodshed', 'military action', 'espionage', 'wuxia action', 'disaster', 'adventure', 'superhero'], 'animation': ['cartoon', 'animation', 'traditional', 'stop motion', 'claymation', 'cutout', 'computer generated imagery', 'puppetry', 'live action'], 'comedy': ['slapstick', 'tragicomedy', 'stoner', 'comedy', 'action comedy', 'dark comedy black comedy', 'romantic comedy', 'buddy comedy', 'road comedy', 'slapstick comedy', 'parody', 'spoof', 'satire', 'sitcom', 'sketch comedy', 'mockumentary', 'prank'], 'crime': ['spy', 'criminal', 'prison', 'slasher', 'whodunit', 'criminal', 'crime', 'caper', 'heist', 'gangster', 'cop police', 'detective', 'courtroom', 'procedural'], 'drama': ['tragicomedy', 'americana', 'tragedy', 'drama', 'melodrama', 'teen drama', 'philosophical drama', 'medical drama', 'legal drama', 'political drama', 'anthropological drama', 'religious drama', 'docudrama'], 'experimental': ['indie',

In [23]:
global i 
i=0

def extract_genres(input_genre_list:object)->list:
    """ returns 1 if one pronoun in pronouns is in headline else 0

    Args:
        pronouns (list): list of pronouns
        headline (object): headline of the article

    Returns:
        list: 1 if there is a pronoun from pronouns in headline else 0 
    """
    genre_list = []
    genres = input_genre_list.copy()
    global i
    i +=1

    if(not i%100):
        print(str(i) +" out of " + str(len(genres_per_year.index)))

    merged_strings = " ".join([str(g) for g in genres]).lower()

    for major_genres, sub_genres in genre_map.items() : 
        for s_g in sub_genres :
            if s_g in merged_strings.split(" "):
                genre_list.append(major_genres)

    if len(genre_list) == 0 :
        genre_list.append("Other")

    genre_list = list(set(genre_list))

    return genre_list

In [24]:
genres_per_year["Major_genres"]=genres_per_year["Movie_genre"].apply(
    partial(extract_genres))

100 out of 74838
200 out of 74838
300 out of 74838
400 out of 74838
500 out of 74838
600 out of 74838
700 out of 74838
800 out of 74838
900 out of 74838
1000 out of 74838
1100 out of 74838
1200 out of 74838
1300 out of 74838
1400 out of 74838
1500 out of 74838
1600 out of 74838
1700 out of 74838
1800 out of 74838
1900 out of 74838
2000 out of 74838
2100 out of 74838
2200 out of 74838
2300 out of 74838
2400 out of 74838
2500 out of 74838
2600 out of 74838
2700 out of 74838
2800 out of 74838
2900 out of 74838
3000 out of 74838
3100 out of 74838
3200 out of 74838
3300 out of 74838
3400 out of 74838
3500 out of 74838
3600 out of 74838
3700 out of 74838
3800 out of 74838
3900 out of 74838
4000 out of 74838
4100 out of 74838
4200 out of 74838
4300 out of 74838
4400 out of 74838
4500 out of 74838
4600 out of 74838
4700 out of 74838
4800 out of 74838
4900 out of 74838
5000 out of 74838
5100 out of 74838
5200 out of 74838
5300 out of 74838
5400 out of 74838
5500 out of 74838
5600 out of 74838
5

C:\Users\nicte\AppData\Local\Temp/ipykernel_1372/544170449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_per_year["Major_genres"]=genres_per_year["Movie_genre"].apply(


In [26]:
pd.set_option('max_colwidth', 100)
genres_per_year.head(20)

,Movie_release,Movie_name,Movie_genre,Major_genres
57963,1888-01-01,Roundhay Garden Scene,"[short film, silent film, indie, black-and-white]","[experimental, black and white, silent]"
38764,1889-01-01,"Leisurely Pedestrians, Open Topped Buses and Hansom Cabs with Trotting Horses","[short film, silent film, indie, black-and-white]","[experimental, black and white, silent]"
60249,1890-01-01,London's Trafalgar Square,"[short film, silent film]","[experimental, silent]"
53907,1890-01-01,"Monkeyshines, No. 1",[silent film],[silent]
7486,1891-01-01,Men Boxing,"[short film, silent film]","[experimental, silent]"
25984,1891-01-01,Dickson Greeting,"[short film, indie]",[experimental]
17849,1891-01-01,Newark Athlete,[short film],[experimental]
50565,1892-10-28,Pauvre Pierrot,"[short film, silent film, indie, animation]","[experimental, silent, animation]"
14231,1892-10-28,Un bon bock,"[short film, silent film, indie, animation]","[experimental, silent, animation]"
39307,1892-10-28,Le Clown et ses chiens,"[short film, silent film, indie, animation]","[experimental, silent, animation]"
